In [4]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import re

In [5]:
filename='report_2016_02_01-2016_12_01'

In [4]:
!ls -l

total 16121840
-rw-rw-r-- 1 sean sean       88495 Jan  3 01:33 crealytics.ipynb
-rw-rw-r-- 1 sean sean         275 Jan  2 22:17 crealytics.py
-rw-rw-r-- 1 sean sean     2750153 Jan  2 22:17 report_2016_02_01-2016_12_01_10000.csv
-rw-rw-r-- 1 sean sean 12575272973 Jan  2 22:11 report_2016_02_01-2016_12_01.csv
-rw-rw-r-- 1 sean sean        8715 Jan  3 01:47 report_2016_02_01-2016_12_01_datadict.csv
-rw-rw-r-- 1 sean sean  3930615902 Jan  2 22:22 report_2016_09_01-2016_12_01.csv


In [2]:
!ls -l

total 3501007
-rwxrwx--- 1 Sean Violante mkpasswd   13680957 Dec  9 12:13 ASOS_UK_bm_report_20161019.xlsx
-rwxrwx--- 1 Sean Violante mkpasswd     849297 Nov 30 14:00 all-reports.csv
-rwxrwx--- 1 Sean Violante mkpasswd     175541 Dec  1 15:10 all-reports.xlsx
-rwxrwx--- 1 Sean Violante mkpasswd     145184 Jan  4 13:11 crealytics.ipynb
-rwxrwx--- 1 Sean Violante mkpasswd        288 Jan  4 13:11 crealytics.py
-rwxrwx--- 1 Sean Violante mkpasswd       9594 Jan  4 13:11 datadict.xlsx
-rwxrwx--- 1 Sean Violante mkpasswd        150 Jan  4 13:12 desktop.ini
-rwxrwx--- 1 Sean Violante mkpasswd    7417274 Jan  4 08:30 lookup_report_2016_09_01-2016_12_01.pkl
-rwxrwx--- 1 Sean Violante mkpasswd 3562083151 Jan  4 08:39 report_2016_09_01-2016_12_01.pkl
-rwxrwx--- 1 Sean Violante mkpasswd      16805 Nov 29 20:24 shopping-performance-report.csv
-rwxrwx--- 1 Sean Violante mkpasswd      12164 Nov 30 13:48 shopping-performance-report.xlsx
-rwxrwx--- 1 Sean Violante mkpasswd        402 Dec  1 14:51 shoppi

In [2]:
df_test=pd.read_csv('report_2016_02_01-2016_12_01.csv.gz',nrows=100000)

In [64]:
reader = pd.read_csv('report_2016_02_01-2016_12_01.csv', chunksize=1000000)

In [28]:
cat_types

array(['Ad group ID', 'Ad group', 'Brand', 'Campaign ID', 'Campaign',
       'Category (1st level)', 'Category (2nd level)',
       'Category (3rd level)', 'Category (4th level)',
       'Category (5th level)', 'Country/Territory', 'Custom label 0',
       'Custom label 1', 'Custom label 2', 'Custom label 3',
       'Custom label 4', 'Device', 'Item Id', 'Product type (1st level)',
       'Product type (2nd level)', 'Product type (3rd level)',
       'Product type (4th level)', 'Product type (5th level)'], dtype=object)

In [29]:
dfs=[]
for chunk in reader:
    df=cast_categ(chunk,cat_types)
    dfs.append(cast_categ)

/home/sean/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (13,26) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/sean/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [6]:

def do_lookup(cat, typ):
    lookups[cat]={}
    
    def lookup( val):
        if val not in lookups[cat]:
            lookups[cat][val]=typ(len(lookups[cat]))
        return lookups[cat][val]
    return lookup

In [7]:
data_dict=pd.read_excel('datadict.xlsx',usecols=range(6), index_col=1)
id_types=data_dict.index[data_dict.data_class=='id'].values
data_dict.loc[data_dict.data_type.notnull(), 'data_type'] = \
    data_dict.loc[data_dict.data_type.notnull(), 'data_type'].map(eval)

In [8]:
create_sqlite = 'CREATE TABLE report_20160201_20161201 ( row_id INTEGER PRIMARY KEY, '
col_def =['{} {}'.format(re.sub('[/\(\.\)]','',tup[0]).replace(' ','_').replace('-','_').lower(), tup[1]) for tup in  data_dict.sqlite.iteritems()]
create_sqlite = create_sqlite + ','.join(col_def) + ')'
create_sqlite

'CREATE TABLE report_20160201_20161201 ( row_id INTEGER PRIMARY KEY, ad_group_id INTEGER,ad_group INTEGER,campaign_id INTEGER,campaign INTEGER,countryterritory INTEGER,day TEXT,device INTEGER,category_1st_level INTEGER,category_2nd_level INTEGER,category_3rd_level INTEGER,category_4th_level INTEGER,category_5th_level INTEGER,product_type_1st_level INTEGER,product_type_2nd_level INTEGER,product_type_3rd_level INTEGER,product_type_4th_level INTEGER,product_type_5th_level INTEGER,custom_label_0 INTEGER,custom_label_1 INTEGER,custom_label_2 INTEGER,custom_label_3 INTEGER,custom_label_4 INTEGER,brand INTEGER,item_id INTEGER,impressions INTEGER,clicks INTEGER,conversions INTEGER,cross_device_conv INTEGER,total_conv_value REAL,cost REAL,click_share TEXT,search_impr_share TEXT,ctr REAL,conv_rate REAL)'

In [17]:
lookups={}
converters={id:do_lookup(id, data_dict.at[id, 'data_type']) for id in id_types  }
converters['CTR'] = lambda(x): np.float32(x[:-1])/100
converters['Conv. rate'] =  lambda(x): np.float32(x[:-1])/100
converters['Total conv. value'] =  lambda(x): np.float32(x.replace(',',''))

parse_dates=['Day']
dtype={ nam: data_dict.at[nam, 'data_type']  for nam in data_dict.index[data_dict.data_type.notnull()]}
# http://stackoverflow.com/a/1176179


 wc report_2016_09_01-2016_12_01.csv 
  14466685  282408421 3930615902 report_2016_09_01-2016_12_01.csv


In [6]:
reader = pd.read_csv('report_2016_09_01-2016_12_01.csv', converters=converters, 
                     parse_dates=parse_dates, dtype=dtype, chunksize=1000000)

In [18]:
reader = pd.read_csv('report_2016_02_01-2016_12_01.csv', converters=converters, 
                     parse_dates=parse_dates, dtype=dtype, chunksize=1000000)

In [92]:
data_dict.index.values

array([u'Ad group ID', u'Ad group', u'Campaign ID', u'Campaign',
       u'Country/Territory', u'Day', u'Device', u'Category (1st level)',
       u'Category (2nd level)', u'Category (3rd level)',
       u'Category (4th level)', u'Category (5th level)',
       u'Product type (1st level)', u'Product type (2nd level)',
       u'Product type (3rd level)', u'Product type (4th level)',
       u'Product type (5th level)', u'Custom label 0', u'Custom label 1',
       u'Custom label 2', u'Custom label 3', u'Custom label 4', u'Brand',
       u'Item Id', u'Impressions', u'Clicks', u'Conversions',
       u'Cross-device conv.', u'Total conv. value', u'Cost',
       u'Click share', u'Search Impr. share', u'CTR', u'Conv. rate'], dtype=object)

In [98]:
!head report_2016_02_01-2016_12_01.csv

Ad group ID,Ad group,Brand,Campaign ID,Campaign,Category (1st level),Category (2nd level),Category (3rd level),Category (4th level),Category (5th level),Clicks,Conv. rate,Conversions,Total conv. value,Cost,Country/Territory,Cross-device conv.,CTR,Custom label 0,Custom label 1,Custom label 2,Custom label 3,Custom label 4,Day,Device,Impressions,Item Id,Product type (1st level),Product type (2nd level),Product type (3rd level),Product type (4th level),Product type (5th level),Click share,Search Impr. share
17601222633,Marken_Michael-Kors,michael kors,201384633,92_Shopping_Marken,Bekleidung & Accessoires,Bekleidung,Bademode, --, --,0,0.00%,0.0,0.0,0,2276,0.0,0.00%, --, --, --, --, --,2016-03-16,Computers,0, --,sport,damen,bade und beach,bikinis mix & match,bikini-tops,< 10%,100.00%
17601222633,Marken_Michael-Kors,michael kors,201384633,92_Shopping_Marken,Bekleidung & Accessoires,Bekleidung,Bademode, --, --,0,0.00%,0.0,0.0,0,2276,0.0,0.00%, --, --, --, --, --,2016-04-13,Computers,0, --,spor

In [97]:
a1.head()

,Ad group ID,Ad group,Brand,Campaign ID,Campaign,Category (1st level),Category (2nd level),Category (3rd level),Category (4th level),Category (5th level),...,Device,Impressions,Item Id,Product type (1st level),Product type (2nd level),Product type (3rd level),Product type (4th level),Product type (5th level),Click share,Search Impr. share
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,< 10%,91.23%
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,< 10%,53.57%
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,1,1,2,1,< 10%,--
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,3,2,< 10%,< 10%
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,3,3,< 10%,87.27%


In [19]:
for i,ch in enumerate(reader):
    print i
    ch=ch[data_dict.index.values]
    ch.to_csv('report_20160201_20161201_star_{:02d}.csv'.format(i),
              index=True,header=False, encoding='utf-8', date_format='%Y-%m-%d')
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


In [9]:
df_mg=pd.concat(dfs)
# need to change to drop original index or have duplicates!

In [ ]:
#need to save lookups

In [11]:
del dfs

In [10]:
df_mg.to_pickle('report_2016_09_01-2016_12_01.pkl')

In [5]:
df_mg=pd.read_pickle('report_2016_09_01-2016_12_01.pkl')

In [18]:
z1=lookups_df.set_index(['id_name', 'id_index'])
cats = z1.index.levels[0].values
for c in cats:
    df_mg[c] = df_mg[c].map(z1.id.xs(c))

In [48]:
df_mg.reset_index(drop=True,inplace=True)

In [53]:
df_mg['Conv. rate'].max()

9.0

In [51]:
df_mg[df_mg['CTR']>1].head().T

,4243,13035,15066,15475,16465
Ad group ID,18790228953,33584868753,33584887833,33584869113,33584881353
Ad group,Marken_Armani,reebok,baldessarini,bogner,shiseido
Brand,giorgio armani cosmetics,reebok,baldessarini,bogner,shiseido
Campaign ID,201384633,644232844,644232844,644232844,644232844
Campaign,92_Shopping_Marken,camato Designer,camato Designer,camato Designer,camato Designer
Category (1st level),Gesundheit & Schönheit,Bekleidung & Accessoires,Gesundheit & Schönheit,Bekleidung & Accessoires,Gesundheit & Schönheit
Category (2nd level),Körperpflege,Bekleidung,Körperpflege,Bekleidung,Körperpflege
Category (3rd level),Kosmetika,Sportbekleidung,Kosmetika,Sportbekleidung,Kosmetika
Category (4th level),Make-up,--,Badeartikel,--,Hautpflege
Category (5th level),Augen,--,Waschlotion,--,Lotion & Feuchtigkeitscremes


In [47]:
df_mg.head()

,Ad group ID,Ad group,Brand,Campaign ID,Campaign,Category (1st level),Category (2nd level),Category (3rd level),Category (4th level),Category (5th level),...,Device,Impressions,Item Id,Product type (1st level),Product type (2nd level),Product type (3rd level),Product type (4th level),Product type (5th level),Click share,Search Impr. share
0,33584867553,sioux,sioux,644232844,camato Designer,Bekleidung & Accessoires,Schuhe,--,--,--,...,Mobile devices with full browsers,1,9079449255937,damen,schuhe,slipper,--,--,--,--
1,33584864193,dsquared2,dsquared2,644232844,camato Designer,Bekleidung & Accessoires,Bekleidung,Hosen,--,--,...,Tablets with full browsers,1,9122948120577,luxus,herren,fashion,jeans,--,--,--
2,33584870313,boss orange,boss orange,644232844,camato Designer,Bekleidung & Accessoires,Bekleidung,Überbekleidung,Mäntel & Jacken,--,...,Computers,8,9100566724609,damen,blazer,klassische blazer,--,--,--,--
3,33584895873,jimmy choo,jimmy choo,644232844,camato Designer,Bekleidung & Accessoires,"Handtaschen, Geldbörsen & Etuis",Handtaschen,--,--,...,Tablets with full browsers,3,9089706590209,luxus,damen,taschen,clutches,--,--,--
4,33584861553,gerry weber,gerry weber,644232844,camato Designer,Bekleidung & Accessoires,Bekleidung,Kleider,--,--,...,Computers,1,9110532128769,damen,kleider,etuikleider,--,--,--,--


In [20]:
desc=df_mg.describe(include='all').T

In [39]:
a=df_mg[df_mg['Clicks']>0].sample(50).copy()

In [46]:
df_mg.loc[42440,'CTR']

42440    0.0
42440    0.0
42440    0.4
42440    0.0
42440    2.0
42440    0.0
42440    0.0
42440    0.0
42440    0.0
42440    0.0
42440    0.0
42440    0.0
42440    0.0
42440    0.0
42440    0.0
Name: CTR, dtype: float64

In [40]:
a['ctr1']=np.round(a['Clicks'].div(a['Impressions']).values,4)

In [41]:
a[['CTR','ctr1']]

,CTR,ctr1
545099,0.0333,0.0333
904670,0.0769,0.0769
946954,0.1667,0.1667
272642,0.5000,0.5000
213827,0.1667,0.1667
155644,0.5000,0.5000
511548,0.0541,0.0541
69952,0.1724,0.1724
613386,0.2222,0.2222
998246,0.1111,0.1111


In [21]:
desc

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
Ad group ID,14466684,2723,33584860953,346002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ad group,14466684,1611,marc o'polo,545346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brand,14466684,1144,marc o'polo,559842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Campaign ID,14466684,6,644232844,8544642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Campaign,14466684,6,camato Designer,8544642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (1st level),14466684,11,Bekleidung & Accessoires,11953652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (2nd level),14466684,35,Bekleidung,9548087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (3rd level),14466684,70,Überbekleidung,3037310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (4th level),14466684,96,--,8073428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (5th level),14466684,56,--,13179366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
desc.to_csv('report_20160901_20161201_desc.csv',encoding='utf-8')

In [16]:
z1.index.levels[0]

Index([u'Ad group', u'Ad group ID', u'Brand', u'Campaign', u'Campaign ID',
       u'Category (1st level)', u'Category (2nd level)',
       u'Category (3rd level)', u'Category (4th level)',
       u'Category (5th level)', u'Country/Territory', u'Custom label 0',
       u'Custom label 1', u'Custom label 2', u'Custom label 3',
       u'Custom label 4', u'Device', u'Item Id', u'Product type (1st level)',
       u'Product type (2nd level)', u'Product type (3rd level)',
       u'Product type (4th level)', u'Product type (5th level)'],
      dtype='object', name=u'id_name')

In [9]:
lookups_df.head()

,id_name,id,id_index
0,Ad group,0039 italy,380
1,Ad group,120%lino,515
2,Ad group,4 barra 12,1373
3,Ad group,4lou,1445
4,Ad group,7 for all mankind,461


In [20]:
df_mg.dtypes

Ad group ID                          int64
Ad group                             int64
Brand                                int64
Campaign ID                          int64
Campaign                             int64
Category (1st level)                 int64
Category (2nd level)                 int64
Category (3rd level)                 int64
Category (4th level)                 int64
Category (5th level)                 int64
Clicks                              uint16
Conv. rate                         float64
Conversions                         uint16
Total conv. value                  float64
Cost                               float32
Country/Territory                    int64
Cross-device conv.                  uint16
CTR                                float64
Custom label 0                       int64
Custom label 1                       int64
Custom label 2                       int64
Custom label 3                       int64
Custom label 4                       int64
Day        

In [20]:
lookups_df1 = pd.concat({id: pd.Series(lookups[id]) for id in lookups}).reset_index()
lookups_df1.columns=['id_name','id', 'id_index']

In [21]:
lookups_df1.head()

,id_name,id,id_index
0,Ad group,0039 italy,1375
1,Ad group,120%lino,944
2,Ad group,365 italy,1820
3,Ad group,4 barra 12,2035
4,Ad group,4lou,1979


In [14]:
lookups_df.to_pickle('lookup_report_2016_09_01-2016_12_01.pkl')

In [22]:
lookups_df1.to_pickle('lookup_report_20160201_20161201.pkl')

In [23]:
lookups_df1.to_csv('lookup_report_20160201_20161201.csv', encoding='utf-8')

In [4]:
lookups_df=pd.read_pickle('lookup_report_2016_09_01-2016_12_01.pkl')

Produce histogram of impressions by each id type
sort by impressions
plot ctr and cvr


In [54]:
def calc_metrics(df,gps):
    met=df.groupby(gps)[['Impressions','Clicks','Conversions', 'Cross-device conv.', 'Total conv. value']].sum()
    met['CTR']=met['Clicks'].div(met['Impressions'])
    met['CVR']=met['Conversions'].div(met['Clicks'])
    met.sort_values('Impressions',ascending=False, inplace=True)
    return met

In [55]:
a1=calc_metrics(df_mg,'Brand')

In [57]:
cat1=calc_metrics(df_mg,'Category (1st level)')

In [62]:
cat1.plot(kind='bar',y=['Impressions','CTR','CVR'],subplots=True)

array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000001D6859E8>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x000000001D909B38>], dtype=object)

In [58]:
cat1.head(20)

,Impressions,Clicks,Conversions,Cross-device conv.,Total conv. value,CTR,CVR
Category (1st level),,,,,,,
Bekleidung & Accessoires,56869009.0,862455.0,12407.0,88.0,3.037757e+06,0.015166,0.014386
Gesundheit & Schönheit,5934291.0,105002.0,3058.0,0.0,2.146530e+05,0.017694,0.029123
Sportartikel,1339465.0,15049.0,226.0,0.0,5.718204e+04,0.011235,0.015018
Heim & Garten,1186823.0,19424.0,339.0,0.0,6.087939e+04,0.016366,0.017453
Taschen & Gepäck,899721.0,11816.0,166.0,0.0,4.235478e+04,0.013133,0.014049
Baby & Kleinkind,38793.0,736.0,20.0,0.0,8.001600e+02,0.018972,0.027174
Bürobedarf,17140.0,321.0,4.0,0.0,8.807800e+02,0.018728,0.012461
Elektronik,2770.0,34.0,0.0,0.0,0.000000e+00,0.012274,0.000000
--,2670.0,35.0,1.0,0.0,8.995000e+01,0.013109,0.028571


In [23]:
lookups_df1=lookups_df.set_index(['id_name','id_index'])

In [59]:
a2=a1.reset_index()

In [60]:
a2['Brand_name']=a2['Brand'].map(lookups_df1.xs('Brand').id)

In [61]:
a2.head()

,Brand,Impressions,Clicks,Conversions,Cross-device conv.,Total conv. value,CTR,CVR,Brand_name
0,19,5653465.0,132052.0,674.0,6.0,186885.390566,0.023358,0.005104,wellensteyn
1,21,2229866.0,36833.0,674.0,0.0,146943.010303,0.016518,0.018299,marc o'polo
2,58,2198560.0,31683.0,452.0,0.0,134021.030163,0.014411,0.014266,hugo
3,63,1895228.0,19856.0,204.0,0.0,118038.810158,0.010477,0.010274,moncler
4,40,1724266.0,24891.0,301.0,0.0,93540.389893,0.014436,0.012093,boss


In [63]:
a2.describe()

/home/sean/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Brand,Impressions,Clicks,Conversions,Cross-device conv.,Total conv. value,CTR,CVR
count,1144.00000,1.144000e+03,1144.000000,1144.000000,1144.000000,1144.000000,1138.000000,1032.000000
mean,571.50000,5.794697e+04,887.131119,14.180070,0.076923,2984.878167,0.016644,inf
std,330.38866,2.481357e+05,4698.710545,48.244519,0.865005,11579.911007,0.015730,NaN
min,0.00000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,285.75000,4.690000e+02,6.000000,0.000000,0.000000,0.000000,NaN,NaN
50%,571.50000,2.987000e+03,45.500000,1.000000,0.000000,77.400002,NaN,NaN
75%,857.25000,2.398975e+04,335.500000,7.000000,0.000000,1514.622499,NaN,NaN
max,1143.00000,5.653465e+06,132052.000000,674.000000,17.000000,186885.390566,0.263158,inf


1144 brands, max 5 million impressions

In [73]:
a2[:50].plot(x='Brand_name',y=['Impressions','CTR','CVR'],subplots=True,kind='bar')

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f8a4fbcdcd0>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x7f8a4f34f5d0>], dtype=object)

In [59]:
import matplotlib.pyplot as plt

In [66]:
f=plt.figure()

In [70]:
z=plt.subplot('311')

In [60]:
%matplotlib

Using matplotlib backend: Qt4Agg


In [53]:
a3

,Brand,Impressions,Clicks,Conversions,Cross-device conv.,Total conv. value,CTR,CVR,Brand_name
0,19,5653465.0,132052.0,674.0,6.0,186885.390566,0.023358,0.005104,wellensteyn
1,21,2229866.0,36833.0,674.0,0.0,146943.010303,0.016518,0.018299,marc o'polo
2,58,2198560.0,31683.0,452.0,0.0,134021.030163,0.014411,0.014266,hugo
3,63,1895228.0,19856.0,204.0,0.0,118038.810158,0.010477,0.010274,moncler
4,40,1724266.0,24891.0,301.0,0.0,93540.389893,0.014436,0.012093,boss
5,8,1666476.0,14575.0,296.0,0.0,64470.710548,0.008746,0.020309,paul green
6,38,1630998.0,19760.0,134.0,8.0,60419.470354,0.012115,0.006781,burberry
7,57,1313391.0,18037.0,106.0,0.0,61218.799752,0.013733,0.005877,woolrich
8,2,1232592.0,19081.0,338.0,0.0,80710.010141,0.015480,0.017714,boss orange
9,13,1200424.0,14882.0,169.0,0.0,21111.800037,0.012397,0.011356,camp david


In [63]:
lookups['Category (1st level)']

{' --': 9,
 'Baby & Kleinkind': 7,
 'Bekleidung & Accessoires': 0,
 'B\xc3\xbcrobedarf': 5,
 'Elektronik': 6,
 'Gesundheit & Sch\xc3\xb6nheit': 1,
 'Heim & Garten': 3,
 'Kunst & Unterhaltung': 8,
 'Religion & Feierlichkeiten': 10,
 'Sportartikel': 2,
 'Taschen & Gep\xc3\xa4ck': 4}

In [73]:
a1.head()

,level_0,level_1,0
0,Ad group,0039 italy,380
1,Ad group,120%lino,515
2,Ad group,4 barra 12,1373
3,Ad group,4lou,1445
4,Ad group,7 for all mankind,461


In [45]:
dtype

{u'Ad group': u'np.uint16',
 u'Ad group ID': u'np.uint16',
 u'Brand': u'np.uint16',
 u'CTR': u'np.float32',
 u'Campaign': u'np.uint16',
 u'Campaign ID': u'np.uint16',
 u'Category (1st level)': u'np.uint16',
 u'Category (2nd level)': u'np.uint16',
 u'Category (3rd level)': u'np.uint16',
 u'Category (4th level)': u'np.uint16',
 u'Category (5th level)': u'np.uint16',
 u'Click share': u'str',
 u'Clicks': u'np.uint16',
 u'Conv. rate': u'np.float32',
 u'Conversions': u'np.uint16',
 u'Cost': u'np.float32',
 u'Country/Territory': u'np.uint16',
 u'Cross-device conv.': u'np.uint16',
 u'Custom label 0': u'np.uint16',
 u'Custom label 1': u'np.uint16',
 u'Custom label 2': u'np.uint16',
 u'Custom label 3': u'np.uint16',
 u'Custom label 4': u'np.uint16',
 u'Device': u'np.uint8',
 u'Impressions': u'np.uint16',
 u'Item Id': u'np.uint32',
 u'Product type (1st level)': u'np.uint16',
 u'Product type (2nd level)': u'np.uint16',
 u'Product type (3rd level)': u'np.uint16',
 u'Product type (4th level)': u'np.

In [44]:
converters

{u'Ad group': <function __main__.lookup>,
 u'Ad group ID': <function __main__.lookup>,
 u'Brand': <function __main__.lookup>,
 'CTR': <function __main__.<lambda>>,
 u'Campaign': <function __main__.lookup>,
 u'Campaign ID': <function __main__.lookup>,
 u'Category (1st level)': <function __main__.lookup>,
 u'Category (2nd level)': <function __main__.lookup>,
 u'Category (3rd level)': <function __main__.lookup>,
 u'Category (4th level)': <function __main__.lookup>,
 u'Category (5th level)': <function __main__.lookup>,
 'Conv. rate': <function __main__.<lambda>>,
 u'Country/Territory': <function __main__.lookup>,
 u'Custom label 0': <function __main__.lookup>,
 u'Custom label 1': <function __main__.lookup>,
 u'Custom label 2': <function __main__.lookup>,
 u'Custom label 3': <function __main__.lookup>,
 u'Custom label 4': <function __main__.lookup>,
 u'Device': <function __main__.lookup>,
 u'Item Id': <function __main__.lookup>,
 u'Product type (1st level)': <function __main__.lookup>,
 u'P

In [5]:
df1=pd.read_csv('report_2016_09_01-2016_12_01.csv', nrows=2000000)

/home/sean/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:

lookups['a']={}
a=do_lookup('a', np.uint16)

In [28]:
a('hello')
a('hi')

1

In [29]:
lookups

{'a': {'hello': 0, 'hi': 1}}

In [6]:
df1['Item Id'].max()

'9142330195969'

In [20]:
def cast_categ(df_inp, cat_types):
    for c in cat_types:
        df_inp[c]=df_inp[c].astype('category')

In [48]:
cast_categ(df1,cat_types)

In [25]:
cnt=df_mg.groupby([ 'Ad group', 'Ad group ID']).size()

In [52]:
cnt.shape

(2576,)

In [26]:
cnt1=cnt.groupby(level='Ad group').size()

In [31]:
cnt.xs('4 barra 12')

Ad group ID
33584830233    48
33584954193     4
dtype: int64

In [27]:
cnt1[cnt1>1]

Ad group
0039 italy                            2
120%lino                              2
4 barra 12                            2
4lou                                  2
7 for all mankind                     2
81hours                               2
a beautiful story                     2
a-k-r-i-s-                            2
abro                                  2
acca kappa                            2
acqua di parma                        2
add                                   2
adenauer                              2
adenauer & co                         2
adidas                                2
adidas by raf simons                  2
adidas by stella mccartney            2
adidas originals                      2
adidas stellasport                    2
aerin                                 2
aeronautica                           2
aeronautica militare                  2
ag jeans                              2
agl attilio giusti leombruni          2
agl-attilio giusti leombruni   

In [38]:
cnt.head()

Ad group         Ad group ID
Marken_Barts     17602163553      125
Marken_Boss      17601240513    12412
Marken_Cambio    17602161273      424
Marken_Desigual  17922688713      174
Marken_Doell     17602162953       57
dtype: int64

In [36]:
cnt.reset_index()['Ad group'].isnull().sum()

0

In [32]:
df_test[['Ad group ID', 'Ad group', 'Brand']].sample(50)

,Ad group ID,Ad group,Brand
99468,17601222753,Marken_Marc-Cain,marccain
9072,17601240513,Marken_Boss,boss
71515,17601229593,Marken_Marc-O'Polo,marc o'polo
24755,17601222753,Marken_Marc-Cain,marccain
75791,17601222753,Marken_Marc-Cain,marccain
32600,17602244913,Marken_Rest,strenesse
22092,17601240513,Marken_Boss,boss
24520,17602244913,Marken_Rest,raffaello rossi
29028,17601230193,Marken_Drykorn,drykorn
94651,17601229593,Marken_Marc-O'Polo,marc o'polo pure


In [10]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
Clicks,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Conversions,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Total conv. value,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Cost,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Country/Territory,100000.0,2276.00000,0.000000,2276.0,2276.0,2276.0,2276.0,2276.0
Cross-device conv.,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Impressions,100000.0,0.00003,0.005477,0.0,0.0,0.0,0.0,1.0


In [7]:
dt

,column_name,data_type
0,Ad group ID,category
1,Ad group,category
2,Brand,category
3,Campaign ID,category
4,Campaign,category
5,Category (1st level),category
6,Category (2nd level),category
7,Category (3rd level),category
8,Category (4th level),category
9,Category (5th level),category
